!pip install periodictable


In [1]:
import pandas as pd
import numpy as np
import re

In [19]:
# read in the various CSVs from the scraping

raw_6 = pd.read_csv('Output Data/Articles_6.csv', encoding = 'utf-8')

raw_6["Stats"]

UnicodeDecodeError: 'ascii' codec can't decode byte 0xc3 in position 18: ordinal not in range(128)

In [3]:
raw = pd.read_csv('Output Data/Articles.csv', encoding = 'utf-8')
raw.head()

,Authors,Citations,Date,Full Text Available,Journal,Stats,Summary,Title
0,"TJ Lu, HA Stone, MF Ashb",567,1998,[PDF] paper.edu.cn,Acta materialia,"TJ Lu, HA Stone, MF Ashby - Acta materialia, 1...",The paper explores the use of open-celled meta...,Heat transfer in open-cell metal foams
1,"K Boomsma, D Poulikakos, Y Ventiko",295,2003,[PDF] psu.edu,… Journal of Heat and Fluid Flow,"K Boomsma, D Poulikakos, Y Ventikos - … Journa...",A new approach to modeling the flow through a ...,Simulations of flow through open cell metal fo...
2,"S Krishnan, JY Murthy",198,2006,[PDF] purdue.edu,Journal of …,"S Krishnan, JY Murthy… - Journal of …, 2006 - ...",Flows in porous media may be modeled using two...,Direct simulation of transport in open-cell me...
3,"A Kopanidis, A Theodorakakos, E Gavaises",141,2010,[PDF] academia.edu,International Journal of …,"A Kopanidis, A Theodorakakos, E Gavaises… - In...",A 3D numerical simulation methodology for the ...,3D numerical simulation of flow and conjugate ...
4,"YW Kwon, RE Cooke, C Par",119,2003,No,Materials Science and Engineering: A,"YW Kwon, RE Cooke, C Park - Materials Science ...",Representative unit-cell models were developed...,Representative unit-cell models for open-cell ...


In [4]:
raw_3 = pd.read_csv('Output Data/Articles_3.csv', encoding = 'utf-8')
raw_3.head()

,Authors,Citations,Date,Full Text Available,Journal,Stats,Summary,Title
0,"L Jing, Z Wang, J Ning, L Zha",73,2011,No,Composites Part B: Engineering,"L Jing, Z Wang, J Ning, L Zhao - Composites Pa...",The deformation and failure modes of dynamical...,The dynamic response of sandwich beams with op...
1,"KC Leong, LW Ji",65,2006,No,International Journal of Heat and fluid flow,"KC Leong, LW Jin - International Journal of He...",An experimental study was performed to investi...,Characteristics of oscillating flow through a ...
2,"WY Jang, S Kyriakides, AM Krayni",102,2010,sciencedirect.com,International Journal of Solids and,"WY Jang, S Kyriakides, AM Kraynik - Internatio...",Two families of finite element models of aniso...,On the compressive strength of open-cell meta...
3,"X Xiao, P Zhang, M L",91,2014,No,International Journal of Thermal Sciences,"X Xiao, P Zhang, M Li - International Journal ...",The thermal conductivity of phase change mater...,Effective thermal conductivity of open-cell me...
4,"KP Dharmasena, HNG Wadle",78,2002,[PDF] virginia.edu,Journal of materials research,"KP Dharmasena, HNG Wadley - Journal of materia...",Cellular metal foams are of interest because o...,Electrical conductivity of open-cell metal foams


In [5]:
## Something wrong with this csv
raw_5 = pd.read_csv('Output Data/Articles_5.csv', encoding = 'utf-8')
raw_5.head()


,Authors,Citations,Date,Full Text Available,Journal,Stats,Summary,Title
0,"A Bhattacharya, VV Calmidi, RL Mahaja",864,2002,No,International Journal of HeatÂ â€¦,"A Bhattacharya, VV Calmidi, RL MahajanÂ - Inte...","â€¦ For these, Îµ varies from 0.3 to 0.6. The ...",Thermophysical properties of high porosity met...
1,"B Jiang, NQ Zhao, CS Shi, JJ L",87,2005,No,Scripta Materialia,"B Jiang, NQ Zhao, CS Shi, JJ LiÂ - Scripta Mat...",â€¦ Metal foams are a new class of materials w...,Processing of open cell aluminum foams with ta...
2,"CY Zhao, SA Tassou, TJ L",111,2008,No,International Journal of Heat and MassÂ â€¦,"CY Zhao, SA Tassou, TJ LuÂ - International Jou...",â€¦ and using it to establish functional relat...,Analytical considerations of thermal radiation...
3,"XH Yang, JJ Kuang, TJ Lu, FS Han",36,2013,[PDF] hfcas.ac.cn,Journal of Physics DÂ â€¦,"XH Yang, JJ Kuang, TJ Lu, FS Hanâ€¦Â - Journal...",We present a simplistic yet accurate analytica...,A simplistic analytical unit cell based model ...
4,"S Feng, Y Zhang, M Shi, T Wen, TJ L",35,2015,[PDF] researchgate.net,Applied Thermal Engineering,"S Feng, Y Zhang, M Shi, T Wen, TJ LuÂ - Applie...",An experimental and theoretical study of the u...,Unidirectional freezing of phase change materi...


In [6]:
## Something wrong with this csv 
raw_4 = pd.read_csv('Output Data/Articles_4.csv', encoding = 'utf-8')
raw_4.head()

,Authors,Citations,Date,Full Text Available,Journal,Stats,Summary,Title
0,"W Lu, CY Zhao, SA Tasso",334,2006,No,International journal of heat and mass transfer,"W Lu, CY Zhao, SA TassouÂ - International jour...","â€¦ 14] will be employed. Although, as indicat...",Thermal analysis on metal-foam filled heat exc...
1,"A Pollien, Y Conde, L Pambaguian",113,2005,[PDF] epfl.ch,Materials Science andÂ â€¦,"A Pollien, Y Conde, L Pambaguianâ€¦Â - Materia...","â€¦ and four-point bend testing), is dictated ...",Graded open-cell aluminium foam core sandwich ...
2,"PJ Tan, SR Reid, JJ Harriga",68,2012,[HTML] sciencedirect.com,International Journal of Solids andÂ â€¦,"PJ Tan, SR Reid, JJ HarriganÂ - International ...",Metal foams are increasingly used for energy a...,[HTML][HTML] On the dynamic mechanical propert...
3,"W Azzi, WL Roberts, A Rabie",60,2007,No,Materials & Design,"W Azzi, WL Roberts, A RabieiÂ - Materials & De...",This research investigated the feasibility of ...,A study on pressure drop and heat transfer in ...
4,"YY Zhao, T Fung, LP Zhang, FL Zhan",147,2005,[PDF] psu.edu,Scripta Materialia,"YY Zhao, T Fung, LP Zhang, FL ZhangÂ - Scripta...","â€¦ absorption, thermal insulation, heat dissi...",Lost carbonate sintering process for manufactu...


# Concatenate the csvs that read well and then start cleaning the data

1. Remove duplicate rows
2. Reset the index
3. Remove rows which have non-numerical data in the Citations column
4. Re-extract the "Date"  from the "Stats" column (some cells were corrupted),  and then change the datatype to int
5. Create Topics_Tags column by parsing the Summary text and Title text and comparing to a list of Topics keywords
6. Create Properties_Tags column by parsing the Summary text and Title text and comparing to a list of Mechanical and Thermal properties keywords
7. Nice to have:  Re-extract the "Authors" and "Journals"  from the Stats column;  there are some spelling errors.

In [7]:
len(raw), len(raw_3), len(raw_5),len(raw_6)

(10, 20, 11, 2611)

In [8]:
# Concatenate the csvs that seem to be working

references = pd.concat([raw,raw_3,raw_5,raw_6])



In [9]:
# Remove duplicate rows

drop_dup = references.drop_duplicates()

len(drop_dup), len(references)

(860, 2652)

In [10]:
# reset the index for ref

ref  = drop_dup.reset_index(drop=True)
len(ref)

860

# Eliminate non-numerics from the Citation Column


In [11]:
# Drop Rows based on whether or not the Citation column contains "% R",  although I couldn't make the wildcards work in this method.
# The below methods were tried and don't work
## STUCK HERE - Trying to eliminate the rows where the Citation number is corrupted (i.e. not a number)
## The pattern which we are trying to get rid of is "4 R", "5 R", "6 R", "7 R", "8 R", "9 R"  in the citation column

# None of the below approaches worked 
#refs_2 = refs_2.drop(refs_1[refs_2.Citations.contains(' R', regex=True)].index)
#refs_2[refs_2.Citations.str.contains('^R'), :] 
#refs_2.loc[refs_2.Citations.str.contains(' R', regex = True), :]

# I was also planning to eliminate the US Patents, from the "Journal" column,  it's more tedious than expected
# https://stackoverflow.com/questions/13851535/how-to-delete-rows-from-a-pandas-dataframe-based-on-a-conditional-expression

#ref1 = ref.drop(ref[ref.Journal=="US Patent 4"].index)
#ref2 = ref1.drop(ref1[ref1.Journal=="US Patent 5"].index)
#ref3 = ref2.drop(ref2[ref2.Journal=="US Patent 6"].index)
#ref4 = ref3.drop(ref3[ref3.Journal=="US Patent 7"].index)
#ref5 = ref4.drop(ref4[ref4.Journal=="US Patent 8"].index)
#ref6 = ref5.drop(ref5[ref5.Journal=="US Patent 9"].index)
#ref7 = ref6.drop(ref6[ref6.Journal=="US Patent"].index)


ref1 = ref.drop(ref[ref.Citations=="4 R"].index)
ref2 = ref1.drop(ref1[ref1.Citations=="5 R"].index)
ref3 = ref2.drop(ref2[ref2.Citations=="6 R"].index)
ref4 = ref3.drop(ref3[ref3.Citations=="7 R"].index)
ref5 = ref4.drop(ref4[ref4.Citations=="8 R"].index)
ref6 = ref5.drop(ref5[ref5.Citations=="9 R"].index)

#len(ref6)

ref6  = ref6.reset_index(drop=True)
ref6["Citations"].head()


ref6




,Authors,Citations,Date,Full Text Available,Journal,Stats,Summary,Title
0,"TJ Lu, HA Stone, MF Ashb",567,1998,[PDF] paper.edu.cn,Acta materialia,"TJ Lu, HA Stone, MF Ashby - Acta materialia, 1...",The paper explores the use of open-celled meta...,Heat transfer in open-cell metal foams
1,"K Boomsma, D Poulikakos, Y Ventiko",295,2003,[PDF] psu.edu,… Journal of Heat and Fluid Flow,"K Boomsma, D Poulikakos, Y Ventikos - … Journa...",A new approach to modeling the flow through a ...,Simulations of flow through open cell metal fo...
2,"S Krishnan, JY Murthy",198,2006,[PDF] purdue.edu,Journal of …,"S Krishnan, JY Murthy… - Journal of …, 2006 - ...",Flows in porous media may be modeled using two...,Direct simulation of transport in open-cell me...
3,"A Kopanidis, A Theodorakakos, E Gavaises",141,2010,[PDF] academia.edu,International Journal of …,"A Kopanidis, A Theodorakakos, E Gavaises… - In...",A 3D numerical simulation methodology for the ...,3D numerical simulation of flow and conjugate ...
4,"YW Kwon, RE Cooke, C Par",119,2003,No,Materials Science and Engineering: A,"YW Kwon, RE Cooke, C Park - Materials Science ...",Representative unit-cell models were developed...,Representative unit-cell models for open-cell ...
5,"N Dukhan, PD Quinon",102,E Cru,[PDF] academia.edu,amos,"N Dukhan, PD Quinones-Ramos, E Cruz-Ruiz… - In...",A one-dimensional heat transfer model for open...,One-dimensional heat transfer analysis in open...
6,"DT Queheillalt, DD Hass, DJ Sypeck",93,2001,[PDF] 128.143.22.36,Journal of Materials …,"DT Queheillalt, DD Hass, DJ Sypeck… - Journal ...","Low-density, open-cell nickel base superalloy ...",Synthesis of open-cell metal foams by template...
7,"B Ozmat, B Leyda, B Benso",82,2004,[PDF] metallicfoams.com,Materials and manufacturing …,"B Ozmat, B Leyda, B Benson - Materials and man...",The key structural and thermo-physical propert...,Thermal applications of open-cell metal foams
8,"M Bai, JN Chun",99,2011,No,International Journal of Thermal Sciences,"M Bai, JN Chung - International Journal of The...",Enhanced cooling methods are needed for advanc...,Analytical and numerical prediction of heat tr...
9,"KK Bodla, JY Murthy, SV Garimell",91,"Part …, 2010",[PDF] purdue.edu,Numerical Heat Transfer,"KK Bodla, JY Murthy, SV Garimella - Numerical ...",Important heat transfer parameters of aluminum...,Microtomography-based simulation of transport ...


In [12]:
# Change datatype for Citations column

ref7 = ref6.astype({"Citations": int, })

In [13]:
len(ref7), ref7.dtypes
#we have dropped out 48 references


(812, Authors                object
 Citations               int32
 Date                   object
 Full Text Available    object
 Journal                object
 Stats                  object
 Summary                object
 Title                  object
 dtype: object)

In [24]:
# Try removing the non-ASciis with python

ref7['Journal']= ref7['Journal'].str.replace('\W+', '')

In [25]:
#Remove the non-standard characters in all columns
#cols = ref7.columns

#ref7[cols] = ref7[cols].replace({
#                                'Ä':'', '¦':'', 'Ÿ':'', '±':'', '–':'', 'Ã':'', 'â':'', '¢':'', 
#                                '¬':'','€':'', 'Ë':'', '†':'', '™':'',
#                               'Œ':'', 'ž':'', 'ƒ':'', '¶':'', '¼':'', 'Å':'', '³':'', 
#                                '¾':'','º':'', '©':'', 'Â':'', 'Â':'',}, regex=True)#

ref7

,Authors,Citations,Date,Full Text Available,Journal,Stats,Summary,Title
0,"TJ Lu, HA Stone, MF Ashb",567,1998,[PDF] paper.edu.cn,Actamaterialia,"TJ Lu, HA Stone, MF Ashby - Acta materialia, 1...",The paper explores the use of open-celled meta...,Heat transfer in open-cell metal foams
1,"K Boomsma, D Poulikakos, Y Ventiko",295,2003,[PDF] psu.edu,JournalofHeatandFluidFlow,"K Boomsma, D Poulikakos, Y Ventikos - … Journa...",A new approach to modeling the flow through a ...,Simulations of flow through open cell metal fo...
2,"S Krishnan, JY Murthy",198,2006,[PDF] purdue.edu,Journalof,"S Krishnan, JY Murthy… - Journal of …, 2006 - ...",Flows in porous media may be modeled using two...,Direct simulation of transport in open-cell me...
3,"A Kopanidis, A Theodorakakos, E Gavaises",141,2010,[PDF] academia.edu,InternationalJournalof,"A Kopanidis, A Theodorakakos, E Gavaises… - In...",A 3D numerical simulation methodology for the ...,3D numerical simulation of flow and conjugate ...
4,"YW Kwon, RE Cooke, C Par",119,2003,No,MaterialsScienceandEngineeringA,"YW Kwon, RE Cooke, C Park - Materials Science ...",Representative unit-cell models were developed...,Representative unit-cell models for open-cell ...
5,"N Dukhan, PD Quinon",102,2005,[PDF] academia.edu,amos,"N Dukhan, PD Quinones-Ramos, E Cruz-Ruiz… - In...",A one-dimensional heat transfer model for open...,One-dimensional heat transfer analysis in open...
6,"DT Queheillalt, DD Hass, DJ Sypeck",93,2001,[PDF] 128.143.22.36,JournalofMaterials,"DT Queheillalt, DD Hass, DJ Sypeck… - Journal ...","Low-density, open-cell nickel base superalloy ...",Synthesis of open-cell metal foams by template...
7,"B Ozmat, B Leyda, B Benso",82,2004,[PDF] metallicfoams.com,Materialsandmanufacturing,"B Ozmat, B Leyda, B Benson - Materials and man...",The key structural and thermo-physical propert...,Thermal applications of open-cell metal foams
8,"M Bai, JN Chun",99,2011,No,InternationalJournalofThermalSciences,"M Bai, JN Chung - International Journal of The...",Enhanced cooling methods are needed for advanc...,Analytical and numerical prediction of heat tr...
9,"KK Bodla, JY Murthy, SV Garimell",91,2010,[PDF] purdue.edu,NumericalHeatTransfer,"KK Bodla, JY Murthy, SV Garimella - Numerical ...",Important heat transfer parameters of aluminum...,Microtomography-based simulation of transport ...


# Extract the Year from the Stats column and change the datatype to integer

There are some errors in the columns,  for Author names, year and journal
https://chrisalbon.com/python/data_wrangling/pandas_regex_to_create_columns/

https://www.geeksforgeeks.org/split-a-string-into-columns-using-regex-in-pandas-dataframe/

In [15]:
# Clean up the Date column  and make the dtype integer

ref7['Date'] = ref7['Stats'].str.extract('(\d\d\d\d)', expand=True) 

ref8 = ref7.astype({"Date": int, })

ref8.dtypes




Authors                object
Citations               int32
Date                    int32
Full Text Available    object
Journal                object
Stats                  object
Summary                object
Title                  object
dtype: object

In [16]:
# Use Python Regex to get the Journal name cleanly from the Stats column
# Also try to get the Authors names cleanly
ref8["Stats"]



0      TJ Lu, HA Stone, MF Ashby - Acta materialia, 1...
1      K Boomsma, D Poulikakos, Y Ventikos - … Journa...
2      S Krishnan, JY Murthy… - Journal of …, 2006 - ...
3      A Kopanidis, A Theodorakakos, E Gavaises… - In...
4      YW Kwon, RE Cooke, C Park - Materials Science ...
5      N Dukhan, PD Quinones-Ramos, E Cruz-Ruiz… - In...
6      DT Queheillalt, DD Hass, DJ Sypeck… - Journal ...
7      B Ozmat, B Leyda, B Benson - Materials and man...
8      M Bai, JN Chung - International Journal of The...
9      KK Bodla, JY Murthy, SV Garimella - Numerical ...
10     L Jing, Z Wang, J Ning, L Zhao - Composites Pa...
11     KC Leong, LW Jin - International Journal of He...
12     WY Jang, S Kyriakides, AM Kraynik - Internatio...
13     X Xiao, P Zhang, M Li - International Journal ...
14     KP Dharmasena, HNG Wadley - Journal of materia...
15     M Vesenjak, C Veyhl, T Fiedler - Materials Sci...
16     DT Queheillalt, Y Katsumura, HNG Wadley - Scri...
17     S Krishnan, SV Garimella

In [18]:
ref8.to_csv("Output Data/reference_ref8.csv", encoding = 'utf-8')
#references.to_csv("Output Data/ref_with_dups.csv", encoding = 'utf-8')
#journals.to_csv("Output Data/journals.csv", encoding = 'utf-8')
#authors.to_csv("Output Data/authors.csv", encoding = 'utf-8')

ref8


,Authors,Citations,Date,Full Text Available,Journal,Stats,Summary,Title,Journals
0,"TJ Lu, HA Stone, MF Ashb",567,1998,[PDF] paper.edu.cn,Acta materialia,"TJ Lu, HA Stone, MF Ashby - Acta materialia, 1...",The paper explores the use of open-celled meta...,Heat transfer in open-cell metal foams,
1,"K Boomsma, D Poulikakos, Y Ventiko",295,2003,[PDF] psu.edu,… Journal of Heat and Fluid Flow,"K Boomsma, D Poulikakos, Y Ventikos - … Journa...",A new approach to modeling the flow through a ...,Simulations of flow through open cell metal fo...,
2,"S Krishnan, JY Murthy",198,2006,[PDF] purdue.edu,Journal of …,"S Krishnan, JY Murthy… - Journal of …, 2006 - ...",Flows in porous media may be modeled using two...,Direct simulation of transport in open-cell me...,
3,"A Kopanidis, A Theodorakakos, E Gavaises",141,2010,[PDF] academia.edu,International Journal of …,"A Kopanidis, A Theodorakakos, E Gavaises… - In...",A 3D numerical simulation methodology for the ...,3D numerical simulation of flow and conjugate ...,
4,"YW Kwon, RE Cooke, C Par",119,2003,No,Materials Science and Engineering: A,"YW Kwon, RE Cooke, C Park - Materials Science ...",Representative unit-cell models were developed...,Representative unit-cell models for open-cell ...,
5,"N Dukhan, PD Quinon",102,2005,[PDF] academia.edu,amos,"N Dukhan, PD Quinones-Ramos, E Cruz-Ruiz… - In...",A one-dimensional heat transfer model for open...,One-dimensional heat transfer analysis in open...,
6,"DT Queheillalt, DD Hass, DJ Sypeck",93,2001,[PDF] 128.143.22.36,Journal of Materials …,"DT Queheillalt, DD Hass, DJ Sypeck… - Journal ...","Low-density, open-cell nickel base superalloy ...",Synthesis of open-cell metal foams by template...,
7,"B Ozmat, B Leyda, B Benso",82,2004,[PDF] metallicfoams.com,Materials and manufacturing …,"B Ozmat, B Leyda, B Benson - Materials and man...",The key structural and thermo-physical propert...,Thermal applications of open-cell metal foams,
8,"M Bai, JN Chun",99,2011,No,International Journal of Thermal Sciences,"M Bai, JN Chung - International Journal of The...",Enhanced cooling methods are needed for advanc...,Analytical and numerical prediction of heat tr...,
9,"KK Bodla, JY Murthy, SV Garimell",91,2010,[PDF] purdue.edu,Numerical Heat Transfer,"KK Bodla, JY Murthy, SV Garimella - Numerical ...",Important heat transfer parameters of aluminum...,Microtomography-based simulation of transport ...,


In [17]:
# Use regex101.com to help setup the proper Regex.  Sometimes it works in Regex and not in Pandas (commented code)



ref8['Journals'] = ref8['Stats'].str.extract(r'-\s+?(\w*\s*\w*\s*){1,10}')

#ref8['Journals'].head(20), len(ref8)



ref8['Journals'] = ref8['Journals'].str.extract(r'-\s+?(\w.*)\,')
print(ref8['Journals'])
# Used the first reference
# https://stackoverflow.com/questions/49184752/iterate-different-regular-expression-patterns-over-df-column
#  https://stackoverflow.com/questions/49169725/combining-for-loop-iterations-into-a-single-line-and-no-match-handling

0       
1       
2       
3       
4       
5       
6       
7       
8       
9       
10      
11      
12      
13      
14      
15      
16      
17      
18      
19      
20      
21      
22      
23      
24      
25      
26      
27      
28      
29      
      ..
782     
783     
784     
785     
786     
787     
788     
789     
790     
791     
792     
793     
794     
795     
796     
797     
798     
799     
800     
801     
802     
803     
804     
805     
806     
807     
808     
809     
810     
811     
Name: Journals, Length: 812, dtype: object


In [ ]:
ref8['Journals'] = ref8['Journals'].replace({
                                'materialia':'Mat.', 'Materials':'Mat.', 'Journal':'J.', 'of':'', 'International':'Intl.', 'Numerical':'Num.', 'Sciences':'Sci.', 
                                'Engineering':'Eng.', 
                                'Science':'Sci','manufacturing':'manufac.', 'and':'', 'Thermal':'Therm.', 'materials':'Mat.',
                                'Materialia':'Mat.', 'Computational':'Comp.','Management':'Mgmt', 'Mechanics':'Mech.', 'Transfer':'Trans',
                                'Transactions': 'Trans', 'Procedia': 'Proc.', 'Industrial':'Indus.', 'engineering':'Eng.', 'journal':'J.',
                                'physica':'Phys', 'status':'stat', 'solidi':'Sol.', 'Experimental': 'Exp', 'Applied':'Appl', 
                                'catalysis':'Cat.', 'environmental':'Env.', 'Physique':'Phys'}, regex=True)

In [ ]:
# Use regex101.com to help setup the proper Regex

#ref8['First Author'] = ref8['Authors'].str.extract(r'(\w+\s+\w+?)\,')
ref8['First Author'] = ref8['Authors'].str.extract(r'(\w+\s\w+\s?\w+)')

ref8['First Author'].head(20)

In [ ]:

ref8.dtypes

# Create lists of keywords to compare to the abstract and title

In [ ]:
#mech_properties = ["Compression Strength","Tensile Strength","Elastic Modulus","Shear Strength","Plasticity"]
#therm_properties = ["Thermal Conductivity","Thermal Resistivity"]
#fluid_properties = ["Permeability","Pressure Drop"]
#elec_properties = ["Electrical Resistivity","Electrical Conductivity","Capacitance"]

#el_table = pd.read_csv('Output Data/periodic_table.csv', encoding = 'utf-8')
#el_table['Name'].apply(lambda x: x.astype(str).str.lower())

#element_name = el_table['Name']
#element_name.str.lower()

#elements = el_table['Sym.']
#compound_names = ["SiC", "silicon carbide", "graphite"]

#elements.reset_index(drop=True, inplace=True)
#element_name.reset_index(drop=True, inplace=True)

#element_name

#print(element_name)
#print(mech_properties)

elements = list(element_name)
elements  = [element.lower() for element in elements]
pattern = "'(" + "|" .join(elements) + ")'"

print(pattern)

ref8["test2"] = ref8['Summary'].str.extract(f'r{pattern}')
print(ref8['test2'])

# Create Topic Tags by searching for adjectives and periodic table elements in  the summary and title


In [ ]:
ref8["mech_prop_sum"] = ref8['Summary'].str.extract(r'(compressive|strength|tensile|elastic|modulus|shear|plasticity)')

ref8["mech_prop_title"] = ref8['Title'].str.extract(r'(compressive|strength|tensile|elastic|modulus|shear|plasticity)')

In [ ]:
ref8["mech_prop_sum"].head(20)


#temp1 = ref8['Summary'].str.extract(r'(compress+)')

#temp2 = ref8['Title'].str.extract(r'(compress+)')

#ref8["Compression 2"].value_counts(),  ref8["Compression Strength"].value_counts()
#ref8["Compression Strength"] = pd.merge(temp1, temp2, how ='outer')
#ref8['Compression Strength']

In [ ]:



test =ref8["mech_prop_sum"].value_counts()
print(test)
testb = ref8["mech_prop_title"].value_counts()
print(testb)




In [ ]:
ref8["elec_prop_sum"] = ref8['Summary'].str.extract(r'(power|resist+|conduct+|electrical conduct+|capacitance|voltage)')
test2 = ref8['elec_prop_sum'].value_counts()
print(test2)

ref8["elec_prop_title"] = ref8['Title'].str.extract(r'(power|resist+|conduct+|electrical conduct+|capacitance|voltage)')
test2b = ref8['elec_prop_title'].value_counts()
print(test2b)

#ref8['elec_prop'] = ref8['elec_prop_sum'] + ref8['elec_prop_title']
#ref8['elec_prop']

In [ ]:
ref8["therm_prop_sum"] = ref8['Summary'].str.extract(r'(thermal conduct+|thermal resist+|heat|temperature)')
test3 = ref8['therm_prop_sum'].value_counts()
print(test3)
ref8["therm_prop_title"] = ref8['Title'].str.extract(r'(thermal conduct+|thermal resist+|heat|temperature)')
test3b = ref8['therm_prop_title'].value_counts()
print(test3b)
#ref8['therm_prop'] = ref8['therm_prop_sum'] + ref8['therm_prop_title']
#ref8['therm_prop']

In [ ]:
ref8["fluid_prop"] = ref8['Summary'].str.extract(r'(permeability|pressure drop)')
test3b = ref8['fluid_prop'].value_counts()
print(test3b)


#ref8['Properties']=ref8["fluid_prop"]+ref8["mech_prop"]+ref8['elec_prop']+ref8['therm_prop']

In [ ]:

ref8["general_sum"]=ref8['Summary'].str.extract(r'(alloy|retic|model)')
test4=ref8['general_sum'].value_counts()
print(test4)

ref8["general_title"]=ref8['Title'].str.extract(r'(alloy|retic|model)')
test4b=ref8['general_title'].value_counts()
print(test4b)

ref8["composition_sum"] = ref8['Summary'].str.extract(r'(SiC|nickel|Ni|titanium|vanadiam|chromium|iron|Fe|Nb|niobium|zirconium|Zr|palladium|Pd|silver|Ag|Tantalum|Ta|platinum|Pt|aluminum|graph|moly|copper|Cu|tungsten|cobalt|gold|Ni|nickel|silicon|carbon)')
test5 = ref8['composition_sum'].value_counts()
print(test5)
ref8['composition_sum']

ref8["composition_title"] = ref8['Title'].str.extract(r'(SiC|nickel|titanium|vanadiam|chromium|iron|Fe|Nb|niobium|zirconium|Zr|palladium|Pd|silver|Ag|Tantalum|Ta|platinum|Pt|aluminum|graphite|moly|copper|Cu|tungsten|cobalt|gold|Ni|nickel|silicon|carbon)')
test5b = ref8['composition_title'].value_counts()
print(test5b)

#ref8['general'] = ref8['general_sum'] + ref8['general_title']
#ref8['general']
#ref8['composition'] = ref8['composition_sum'] + ref8['composition_title']
#ref8['composition']

In [ ]:
ref8.to_csv("Output Data/reference_abbr.csv", encoding = 'utf-8')
#references.to_csv("Output Data/ref_with_dups.csv", encoding = 'utf-8')
#journals.to_csv("Output Data/journals.csv", encoding = 'utf-8')
#authors.to_csv("Output Data/authors.csv", encoding = 'utf-8')

ref8


# Prepare the Journal and Author data for plotting in Chart.js

In [ ]:
journal_pareto = ref8["Journals"].value_counts()
#journal_pareto.head(10)
journals = journal_pareto.iloc[:10,]
journals



In [ ]:
Author_pareto = ref8["First Author"].value_counts()
#Author_pareto. head(10)
authors = Author_pareto[:10,]
authors